In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from agent_api.agent import GroqAgent

In [3]:
agent = GroqAgent(
    name="my_test_agent",
    mcp_server="http://127.0.0.1:9090/mcp",
    active_tools=["process_video", "get_video_clip_from_image"],
)
await agent.setup()

2025-06-09 13:16:31.834 | INFO     | agent_api.agent.base_agent:discover_tools:86 - Discovered 4 tools:
2025-06-09 13:16:31.835 | INFO     | agent_api.agent.base_agent:discover_tools:88 - Filtered tools to 2 active tools
2025-06-09 13:16:31.835 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - get_video_clip_from_user_query: Use this tool to get a video clip from a video file based on a user query or question.
2025-06-09 13:16:31.835 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - ask_question_about_video: Use this tool to get an answer to a question about the video.
2025-06-09 13:16:31.835 | INFO     | agent_api.agent.base_agent:_get_routing_system_prompt:41 - Getting routing system prompt


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/moteroperdido/.pixeltable/pgdata


2025-06-09 13:16:32.314 | INFO     | agent_api.agent.base_agent:_get_tool_use_system_prompt:47 - Getting tool use system prompt
2025-06-09 13:16:32.446 | INFO     | agent_api.agent.base_agent:_get_general_system_prompt:53 - Getting general system prompt


In [4]:
print(agent.routing_system_prompt)


You are a routing assistant part of a video processing application. 
You need to determine if the user query requires the use of a tool. 
The tools available are:

- get_video_clip_from_user_query: This tool is used to get a clip from the video based on the user query.
- ask_question_about_video: This tool is used to ask a question about the video.

You output should be a boolean value indicating it tool use is required or not.



In [5]:
print(agent.tool_use_system_prompt)


Your name is Kubrick.
You are a tool use assistant part of a video processing application.
You need to determine which tool to use based on the user query (if any).

The tools available are:

- get_video_clip_from_user_query: This tool is used to get a clip from the video based on the user query.
- ask_question_about_video: This tool is used to get some information about the video.

The video path is: {video_path}



In [6]:
print(agent.general_system_prompt)


Your name is Kubrick. 
You are a general assistant part of a video processing application.
You need to help the user with their query.



In [7]:
agent._route_query("Hello, I'm Miguel!")

False

In [8]:
agent._route_query("Get me the clip when the character says 'Hello, I'm Miguel!'")

True

In [11]:
await agent.chat("Yoooo", "./videos/pass_the_butter_rick_and_morty.mp4")

Inserting rows into `memory`: 1 rows [00:00, 53.63 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 413.56 rows/s]
Inserted 1 row with 0 errors.


"I'm ready when you are! Please go ahead and upload the video to the server. Once it's uploaded, I'll do my best to analyze the video and tell you what the color of the table is."

In [9]:
await agent.chat(
    "In the video, can you tell me what is the color of the table?",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

Inserting rows into `memory`: 1 rows [00:00, 73.13 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 275.94 rows/s]
Inserted 1 row with 0 errors.


'The color of the table in the video is yellow and green plaid, as indicated by the tablecloth.'

In [10]:
await agent.chat(
    "Give me a clip of the video where Rick says 'Pass the butter'",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

Inserting rows into `memory`: 1 rows [00:00, 202.47 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 870.91 rows/s]
Inserted 1 row with 0 errors.


'Here is the clip of the video where Rick says "Pass the butter". The clip has been saved as ./videos/d8163345-7d9a-4909-91ab-71443b513813.mp4.'

In [11]:
agent.reset_memory()

2025-06-09 13:18:21.742 | INFO     | agent_api.agent.memory:reset_memory:37 - Resetting memory: my_test_agent
